In [115]:
import pandas as pd

In [ ]:
olist_customers = pd.read_csv("olist_customers_dataset.csv", sep=",", usecols=['customer_id', 'customer_unique_id'])
olist_orders = pd.read_csv("olist_orders_dataset.csv", sep=",", usecols=['customer_id', 'order_id', 'order_status', 'order_purchase_timestamp'])
olist_order_reviews = pd.read_csv("olist_order_reviews_dataset.csv", sep=",", usecols=['order_id', 'review_score'])
olist_order_items = pd.read_csv("olist_order_items_dataset.csv", sep=",", usecols=['order_id', 'product_id'])

# Verificando a volumetria de pedidos e produtos de cada pedido

Dentro da base de "olist_order_items", contém o order_ir e o product_id de cada compra.

Abaixo é uma verificação para entender quais produtos tinham dentro de um pedidos e se eram diferentes.

Isso ajudará entender se é possível usar essa base para gerar o modelo de recomendação de produtos.

In [117]:
produtos = olist_order_items.groupby('order_id').agg(total=('product_id','count'),
                                          distinto=('product_id','nunique'))
produtos.reset_index(inplace=True)

In [118]:
produtos_df = pd.DataFrame({
    'volumetria_pedidos':produtos['total'].value_counts(),
    'volumetria_pedidos_perc':(produtos['total'].value_counts(normalize=True)*100).round(2),
})
produtos_df.head()

,volumetria_pedidos,volumetria_pedidos_perc
total,,
1,88863,90.06
2,7516,7.62
3,1322,1.34
4,505,0.51
5,204,0.21


In [119]:
produtos_df = pd.DataFrame({
    'volumetria_disintos':produtos['distinto'].value_counts(),
    'volumetria_disintos_perc':(produtos['distinto'].value_counts(normalize=True)*100).round(2),
})
produtos_df.head()

,volumetria_disintos,volumetria_disintos_perc
distinto,,
1,95430,96.72
2,2846,2.88
3,298,0.30
4,70,0.07
6,10,0.01


90% da base consiste em pedidos feitos que contém apenas um produto e 7% que contém 2 produtos.

96% dos pedidos tem o mesmo produto enquanto 4% possuiam produtos diferentes.

In [120]:
lista_de_pedidos_com_unico_produto = produtos[produtos['distinto'] == 1]['order_id']

Como 96% da base consiste em pedidos com o mesmo produto, será possível utilizar o order_id como uma proxy para o produto, já que as bases da olist não possuem a avalição feita por produto, mas apenas por pedido.

# Verificando a volumetria de notas para cada pedido

In [121]:
#Filtrando os order_id que estão dentro da lista de pedidos com único tipo de produto
df_produtos = olist_orders[olist_orders['order_id'].isin(lista_de_pedidos_com_unico_produto)]

#Comparando a qtd de linhas
df_produtos.shape[0], olist_orders.shape[0]

(95430, 99441)

<span style="font-size:24px">**Filtro de Dados Utilizados no Modelo de Recomendação**</span>

Filtrando a base apenas pelos pedidos entregues para evitar qualquer ruído que possa gerar de outros status que possam ter avaliações indevidas aos produtos.

In [122]:
df_produtos = df_produtos[df_produtos['order_status'] == 'delivered'][['customer_id', 'order_id']]

Na base de Usuários da Olist (olist_customers) existem dois ids de usuários um único e outro não, vamos cruzar essa base com a de pedidos apenas para utilizar o id único no modelo no futuro.

In [123]:
#Cruzamentos
df_produtos_avaliacoes = df_produtos.merge(olist_customers, on="customer_id", how="inner") \
    .merge(olist_order_reviews, on="order_id", how='left')
df_produtos_avaliacoes.drop(columns='customer_id', inplace=True)
df_produtos_avaliacoes.head()

,order_id,customer_unique_id,review_score
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,4.0
1,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,4.0
2,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8,5.0
3,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977,5.0
4,ad21c59c0840e6cb83a9ceb5573f8159,72632f0f9dd73dfee390c9b22eb56dd6,5.0


In [124]:
# Verificando se há valores vazios
df_produtos_avaliacoes.isna().value_counts()

order_id  customer_unique_id  review_score
False     False               False           93163
                              True              608
Name: count, dtype: int64

In [125]:
#Filtrando os pedidos que estão sem avaliações
df_produtos_avaliacoes.dropna(subset=['review_score'], inplace=True)

In [126]:
# Renomeando o nome das colunas
df_produtos_avaliacoes.rename(columns={
    'customer_unique_id':'user_id',
    'order_id':'product_id',
    'review_score':'rating'
}, inplace=True)

In [127]:
#Comparando a quantidade de usuários e comparação a quantidade de avalições feitas
avaliacoes_por_user_id = df_produtos_avaliacoes['user_id'].value_counts()
usuarios_por_qtd_avaliacoes = avaliacoes_por_user_id.value_counts().sort_index()
usuarios_por_qtd_avaliacoes_df = pd.DataFrame({
    'qtd_avaliacoes': usuarios_por_qtd_avaliacoes.index,
    'qtd_usuarios': usuarios_por_qtd_avaliacoes.values,
    'qtd_usuarios_perct':((usuarios_por_qtd_avaliacoes.values / usuarios_por_qtd_avaliacoes.values.sum()) * 100).round(2)
})
usuarios_por_qtd_avaliacoes_df

,qtd_avaliacoes,qtd_usuarios,qtd_usuarios_perct
0,1,87323,97.11
1,2,2154,2.40
2,3,310,0.34
3,4,101,0.11
4,5,22,0.02
5,6,11,0.01
6,7,1,0.00
7,15,1,0.00


In [128]:
df_produtos_avaliacoes.head()

,product_id,user_id,rating
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,4.0
1,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,4.0
2,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8,5.0
3,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977,5.0
4,ad21c59c0840e6cb83a9ceb5573f8159,72632f0f9dd73dfee390c9b22eb56dd6,5.0


In [129]:
usuarios = df_produtos_avaliacoes.groupby('user_id').agg(pedidos=('product_id','count'),
                                          avaliacoes=('rating','count'),
                                          avaliacoes_avg=('rating','mean'))
usuarios.reset_index(inplace=True)
usuarios

,user_id,pedidos,avaliacoes,avaliacoes_avg
0,0000366f3b9a7992bf8c76cfdf3221e2,1,1,5.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,1,4.0
2,0000f46a3911fa3c0805444483337064,1,1,3.0
3,0000f6ccb0745a6a4b88665a16c9f078,1,1,4.0
4,0004aac84e0df4da2b147fca70cf8255,1,1,5.0
...,...,...,...,...
89918,fffbf87b7a1a6fa8b03f081c5f51a201,1,1,5.0
89919,fffea47cd6d3cc0a88bd621562a9d061,1,1,4.0
89920,ffff371b4d645b6ecea244b27531430a,1,1,5.0
89921,ffff5962728ec6157033ef9805bacc48,1,1,5.0


In [130]:
usuarios_df = pd.DataFrame({
    'volumetria_usuarios':usuarios['pedidos'].value_counts(),
    'volumetria_usuarios_perc':(usuarios['pedidos'].value_counts(normalize=True)*100).round(2)
})
usuarios_df.head()

,volumetria_usuarios,volumetria_usuarios_perc
pedidos,,
1,87323,97.11
2,2154,2.40
3,310,0.34
4,101,0.11
5,22,0.02


In [131]:
usuarios_df = pd.DataFrame({
    'volumetria_usuarios':usuarios['avaliacoes_avg'].value_counts(),
    'volumetria_usuarios_perc':(usuarios['avaliacoes_avg'].value_counts(normalize=True)*100).round(2)
})
usuarios_df.head()

,volumetria_usuarios,volumetria_usuarios_perc
avaliacoes_avg,,
5.0,53346,59.32
4.0,17830,19.83
1.0,8154,9.07
3.0,7369,8.19
2.0,2609,2.90


Na base de dados, aproximadamente 97% do usuários realizaram apenas um pedido no período.

A maior parte dos usuários, em seus únicos pedidos entregues, deram notas altas para os produtos. Apenas aproximadamente 12% deram uma nota baixa (1 ou 2) e 8% deram uma nota mediana.

In [132]:
df_produtos_avaliacoes['user_mean'] = df_produtos_avaliacoes.groupby('user_id')['rating'].transform('mean')
df_produtos_avaliacoes['user_reviews_count'] = df_produtos_avaliacoes.groupby('user_id')['rating'].transform('count')
df_produtos_avaliacoes['normalized_rating'] = df_produtos_avaliacoes['rating'] - df_produtos_avaliacoes['user_mean']

Mesmo com 97% dos usuários tendo um pedido, vamos no modelo testar tanto o valor absoluto da média de notas quanto os valores normalizado.

In [134]:
df_produtos_avaliacoes.to_excel('df_produtos_avaliacoes.xlsx')

# Resumo da Análise Exploratória

- 90% dos pedidos são conspostos de um único item e quando tem mais de um item.
- 96% dos pedidos tem um único produto ou produtos iguais.
- Possível utilziar o **order_id** como um proxy para o **product_id**, já que nas bases da Olist não há avaliações por produto, apenas por pedidos realizados.
- 97% dos usuários realizaram apenas um pedido e o avaliaram.
- Considerando uma escala de 1 a 5: aproximadamente 12% dos usuários deram uma nota baixa para o produto (1 ou 2), 8% deram uma nota mediana (3) e 80% deram uma nota alta (4 ou 5).
- Mesmo com 97% dos usuários tendo um pedido, vamos no modelo testar tanto o valor absoluto da média de notas quanto os valores normalizado. 